In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,Target
0,0.989,0.344,0.844,0.289,Female,0.578,0.100,Professor,0.811,1.078,Poor,0.156,2.0
1,0.600,0.244,0.167,0.133,Male,0.911,0.489,Lecturer,0.122,0.311,Rich,0.711,4.0
2,0.011,0.600,0.533,0.189,Female,0.456,0.178,Lecturer,0.444,0.789,Poor,0.144,3.0
3,0.233,0.333,0.056,0.989,Female,0.733,0.089,Lecturer,0.056,0.744,Rich,1.022,2.0
4,0.722,0.489,1.089,1.089,Female,0.200,0.356,Lecturer,0.544,0.822,Poor,0.156,2.0


In [3]:

df = df.iloc[df.isnull().sum(1).sort_values(ascending=0).index]
df.isnull().sum(axis=1)


694    13
624    12
596    11
317    11
316    11
       ..
248     0
249     0
250     0
251     0
699     0
Length: 700, dtype: int64

In [4]:
cols = len(df.axes[1])
df = df[df.isnull().sum(axis=1) < cols / 2.0]
df.isnull().sum(axis=1)

200    5
199    5
198    5
197    5
196    5
      ..
248    0
249    0
250    0
251    0
699    0
Length: 683, dtype: int64

In [5]:

columns = ['F1', 'F2', 'F3', 'F4', 'F6', 'F7', 'F9', 'F10', 'F12', 'Target']

for column in columns:
  df[column] = df[column].fillna(df[column].median())

df.isnull().sum()

F1        0
F2        0
F3        0
F4        0
F5        0
F6        0
F7        0
F8        0
F9        0
F10       0
F11       0
F12       0
Target    0
dtype: int64

In [6]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

columns = ['F5', 'F8', 'F11']

for column in columns:
  df[column] = label_encoder.fit_transform(df[column])
  df[column].unique()

df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,Target
200,0.711,0.689,1.089,1.111,0,0.533,0.556,1,0.567,0.578,0,0.544,2.0
199,0.544,0.467,0.067,0.256,1,0.533,0.556,0,0.567,0.578,1,0.544,4.0
198,1.044,0.667,0.256,0.811,0,0.533,0.556,0,0.567,0.578,1,0.544,3.0
197,0.967,1.100,1.078,0.044,0,0.533,0.556,0,0.567,0.578,0,0.544,5.0
196,0.067,1.033,0.511,0.744,1,0.533,0.556,0,0.567,0.578,1,0.544,1.0


In [7]:

df['Target'].unique()

array([2., 4., 3., 5., 1.])

In [9]:
# importing the module
import numpy as np

# displaying the datatypes
display(df.dtypes)

# converting 'Field_2' from float to int
df['Target'] = df['Target'].apply(np.int64)


F1        float64
F2        float64
F3        float64
F4        float64
F5          int64
F6        float64
F7        float64
F8          int64
F9        float64
F10       float64
F11         int64
F12       float64
Target      int64
dtype: object

In [10]:
# importing the module
import numpy as np

# displaying the datatypes
display(df.dtypes)

# converting 'Field_2' from float to int
df['Target'] = df['Target'].apply(np.int64)

F1        float64
F2        float64
F3        float64
F4        float64
F5          int64
F6        float64
F7        float64
F8          int64
F9        float64
F10       float64
F11         int64
F12       float64
Target      int64
dtype: object

In [11]:
from collections import Counter
counter = Counter(df['Target'])
print(counter)

Counter({4: 153, 2: 141, 5: 135, 1: 128, 3: 126})


In [12]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(df.drop('Target', axis=1), df['Target'])

In [13]:

counter = Counter(y)
print(counter)

Counter({2: 153, 4: 153, 3: 153, 5: 153, 1: 153})


In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
principal_components = pca.fit_transform(X_scaled)
principal_components.shape

(765, 8)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(principal_components,y, test_size=0.25, stratify=y, random_state=42)


In [20]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=4, random_state=0)

In [21]:

clf.fit(X_train, Y_train)

RandomForestClassifier(max_depth=4, random_state=0)

In [22]:
pre = clf.predict(X_test)

In [23]:
from sklearn.metrics import classification_report

target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
print(classification_report(Y_test, pre, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.23      0.18      0.20        39
     class 2       0.21      0.18      0.20        38
     class 3       0.26      0.32      0.29        38
     class 4       0.17      0.18      0.18        39
     class 5       0.26      0.29      0.28        38

    accuracy                           0.23       192
   macro avg       0.23      0.23      0.23       192
weighted avg       0.23      0.23      0.23       192



In [24]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, Y_train)
pre = clf.predict(X_test)
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
print(classification_report(Y_test, pre, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.23      0.18      0.20        39
     class 2       0.20      0.21      0.20        38
     class 3       0.29      0.42      0.34        38
     class 4       0.16      0.13      0.14        39
     class 5       0.30      0.26      0.28        38

    accuracy                           0.24       192
   macro avg       0.23      0.24      0.23       192
weighted avg       0.23      0.24      0.23       192



In [25]:
from sklearn.ensemble import ExtraTreesClassifier
clf=ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, Y_train)
pre = clf.predict(X_test)
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
print(classification_report(Y_test, pre, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.31      0.36      0.33        39
     class 2       0.26      0.24      0.25        38
     class 3       0.24      0.26      0.25        38
     class 4       0.09      0.08      0.08        39
     class 5       0.24      0.24      0.24        38

    accuracy                           0.23       192
   macro avg       0.23      0.23      0.23       192
weighted avg       0.23      0.23      0.23       192



In [26]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(estimator=SVC(),n_estimators=10, random_state=0)
clf.fit(X_train, Y_train)
pre = clf.predict(X_test)
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
print(classification_report(Y_test, pre, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.25      0.26      0.25        39
     class 2       0.28      0.26      0.27        38
     class 3       0.25      0.37      0.30        38
     class 4       0.23      0.15      0.18        39
     class 5       0.29      0.26      0.28        38

    accuracy                           0.26       192
   macro avg       0.26      0.26      0.26       192
weighted avg       0.26      0.26      0.26       192



In [27]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, Y_train)
pre = clf.predict(X_test)
from sklearn.metrics import classification_report
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5']
print(classification_report(Y_test, pre, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.16      0.21      0.18        39
     class 2       0.21      0.26      0.23        38
     class 3       0.20      0.16      0.18        38
     class 4       0.15      0.13      0.14        39
     class 5       0.20      0.16      0.18        38

    accuracy                           0.18       192
   macro avg       0.18      0.18      0.18       192
weighted avg       0.18      0.18      0.18       192



In [31]:
from sklearn.ensemble import AdaBoostRegressor
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, Y_train)
regr.score(X_test,Y_test)

-0.00352262633291045